In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets

In [2]:
root_folder = '/kaggle/input/data'
file_paths_dict = {}
for root, dirs, files in os.walk(root_folder):
    if root != root_folder:
        for f in files:
            file_name = f
            file_paths_dict[file_name] = os.path.join(root, f)
            
print(file_paths_dict["00006199_010.png"])

/kaggle/input/data/images_003/images/00006199_010.png


In [3]:
all_df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')

all_df['path'] = all_df['Image Index'].map(file_paths_dict.get)
all_df['path'] = all_df['Image Index'].map(file_paths_dict.get)

all_df.sample(3)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
6951,00001837_000.png,No Finding,0,1837,40,M,PA,2048,2500,0.171,0.171,NaN,/kaggle/input/data/images_002/images/00001837_...
7051,00001860_003.png,No Finding,3,1860,47,F,PA,2992,2991,0.143,0.143,NaN,/kaggle/input/data/images_002/images/00001860_...
32189,00008423_001.png,No Finding,1,8423,55,M,PA,2992,2991,0.143,0.143,NaN,/kaggle/input/data/images_004/images/00008423_...


In [4]:
def convert_label(label):
    if label == "No Finding":
        return 0
    else:
        return 1

all_df['Finding Labels'] = all_df['Finding Labels'].apply(convert_label)

all_df.sample(3)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
46442,00011868_003.png,1,3,11868,23,M,PA,2992,2991,0.143000,0.143000,NaN,/kaggle/input/data/images_006/images/00011868_...
96679,00025476_005.png,0,5,25476,36,M,AP,3056,2544,0.139000,0.139000,NaN,/kaggle/input/data/images_011/images/00025476_...
107342,00028974_019.png,0,19,28974,77,M,PA,1631,2021,0.194311,0.194311,NaN,/kaggle/input/data/images_012/images/00028974_...


In [5]:
df = all_df.sample(4000)
df = df.reset_index(drop=True)

value_counts = df['Finding Labels'].value_counts()
print(value_counts)

df.head()

Finding Labels
0    2184
1    1816
Name: count, dtype: int64


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
0,00013644_000.png,0,0,13644,19,F,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_006/images/00013644_...
1,00019208_005.png,1,5,19208,44,M,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_009/images/00019208_...
2,00001758_000.png,0,0,1758,41,M,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_002/images/00001758_...
3,00010198_015.png,1,15,10198,66,M,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_005/images/00010198_...
4,00020146_003.png,0,3,20146,22,M,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_009/images/00020146_...


In [6]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):

        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataframe.iloc[idx, 12]
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define your transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CustomImageDataset(dataframe=df_train, transform=transform)
test_dataset = CustomImageDataset(dataframe=df_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:

class VGG19(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG19, self).__init__()
        self.features = self._make_layers([
            64, 64, 'M', 
            128, 128, 'M', 
            256, 256, 256, 256, 'M', 
            512, 512, 512, 512, 'M', 
            512, 512, 512, 512, 'M'
        ])
        self.classifier = nn.Sequential(
            nn.Linear(8192, 4096),  # Adjusted to match the actual size
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _make_layers(self, cfg):
        layers = []
        in_channels = 1  # Change to 3 if your images are RGB
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

# Initialize the model, loss function, and optimizer
model = VGG19(num_classes=2)  # For binary classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Assuming 'train_loader' and 'test_loader' are defined as in your simple CNN example

# Training Loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

    # Validation Loop
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on validation set: {100 * correct / total}%')

Epoch 1, Loss: 0.6976352250576019
Accuracy on validation set: 52.125%
Epoch 2, Loss: 0.6887110435962677
Accuracy on validation set: 52.125%
Epoch 3, Loss: 0.6886497640609741
Accuracy on validation set: 52.125%
Epoch 4, Loss: 0.6888317114114761
Accuracy on validation set: 52.125%
Epoch 5, Loss: 0.6886330312490463
Accuracy on validation set: 52.125%
Epoch 6, Loss: 0.6883176553249359
Accuracy on validation set: 52.125%
Epoch 7, Loss: 0.6891204041242599
Accuracy on validation set: 52.125%
Epoch 8, Loss: 0.6881031328439713
Accuracy on validation set: 52.125%
Epoch 9, Loss: 0.6880787473917007
Accuracy on validation set: 52.125%
Epoch 10, Loss: 0.6881593996286393
Accuracy on validation set: 52.125%
Epoch 11, Loss: 0.6885913383960723
Accuracy on validation set: 52.125%
Epoch 12, Loss: 0.6886723417043686
Accuracy on validation set: 52.125%
Epoch 13, Loss: 0.6883285057544708
Accuracy on validation set: 52.125%
Epoch 14, Loss: 0.6882947736978531
Accuracy on validation set: 52.125%
Epoch 15, Loss: